In [1]:
from parse_data import parse_file
import numpy as np
import os

In [2]:
def compute_demand(r, demand):
    return sum(demand[n+1] for n in r)

def compute_cost(r, coords):
    if r[0] != 0:
        r = [0] + r
    if r[-1] != 0:
        r = r + [0]
    return np.sum([np.sqrt(np.sum((coords[n+1] - coords[p+1])**2)) for p, n in zip(r[:-1], r[1:])])

def path(file):
    return os.path.join("/home/ubuntu/dev", file)

## CMT1

In [3]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT1.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [4]:
solution = """46 5 49 10 39 33 45 15 44 37 12
11 2 29 21 16 50 34 30 9 38
8 26 31 28 3 36 35 20 22 1 32
27 48 23 7 43 24 25 14 6
18 13 41 40 19 42 17 4 47"""
# Cost 524.611

In [5]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [6]:
len(sub_tours)

5

In [7]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

524.6111466425073

### LKH3 Solution

In [8]:
tour = [0, 11, 2, 29, 21, 16, 50, 34, 30, 9, 38, 0, 46, 5, 
        49, 10, 39, 33, 45, 15, 44, 37, 12, 0, 8, 26, 31, 
        28, 3, 36, 35, 20, 22, 1, 32, 0, 47, 4, 17, 42, 
        19, 40, 41, 13, 18, 0, 6, 14, 25, 24, 43, 7, 23, 48, 27, 0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 524.6111466425073
Gap: 0.0%


#### Duration
Average serial duration: 12.696086645126343

### OR-Tools

In [9]:
tour_str = """
0 27 1 28 3 36 35 20 29 21 50 16 38 0
0 46 12 17 37 44 42 19 40 41 13 4 0
0 47 18 25 14 24 43 48 0
0 5 49 9 34 30 10 39 33 45 15 0
0 6 23 7 26 8 31 22 2 11 32 0"""

sub_tours = [[int(n) for n in r.split()[1:-1]] for r in tour_str.split("\n") if r]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 556.4565079224349
Gap: 6.070279193215162%


#### Duration
Average serial duration: 60.0

### RL current solution

In [10]:
tour = [ 0,  6, 14, 25, 24, 43,  7, 23, 48, 27,  0, 18, 13, 41, 40, 19, 42,  4,
        47,  0, 12, 17, 37, 44, 15, 45, 33, 39, 10, 49,  5,  0, 38,  9, 30, 34,
        50, 21, 29, 16,  2, 11,  0, 32, 20, 35, 36,  3, 28, 31, 26,  8, 22,  1,
        46,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 531.8638624750281
Gap: 1.3824936581957814%


#### Duration
Average serial duration: 1.5238912

In [11]:
tour = [ 0,  6, 25, 14, 24, 43,  7, 23, 27, 48,  0, 18, 13, 41, 40, 42, 19,  4,
        47,  0, 12, 17, 37, 44, 15, 45, 33, 39, 10, 49,  5,  0, 38,  9, 30, 34,
        50, 21, 29,  2, 16, 11,  0, 32, 35, 36, 20,  3, 28, 31, 26,  8, 22,  1,
        46,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 562.1077232366748
Gap: 7.147499025543835%


#### Duration
Average serial duration: 7.2

## CMT2

In [12]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT2.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [13]:
solution = """68 2 28 61 21 74 30
53 66 65 38 10 58
62 22 64 42 41 43 1 73 51
48 47 36 69 71 60 70 20 37 27
26 12 39 9 40 17
3 44 32 50 18 55 25 31 72
45 29 5 15 57 13 54 19 52
7 11 59 14 35 8
75 4 34 46 67
6 33 63 23 56 24 49 16"""
# Cost 835.262

In [14]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [15]:
len(sub_tours)

10

In [16]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

835.2621066076043

### LKH3 Solution

In [17]:
tour = [0, 45, 29, 5, 15, 57, 13, 54, 19, 52, 0, 8, 35, 14, 59, 53, 
        7, 26, 0, 12, 72, 39, 9, 32, 44, 3, 17, 0, 6, 33, 63, 23, 
        56, 24, 49, 16, 0, 30, 74, 21, 61, 28, 2, 68, 0, 62, 22, 
        64, 42, 41, 43, 1, 73, 51, 0, 10, 31, 25, 55, 18, 50, 40, 
        0, 27, 37, 20, 70, 60, 71, 69, 36, 47, 48, 0, 11, 66, 65, 
        38, 58, 0, 75, 4, 34, 46, 67, 0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 10
Total clients visited: 75
Total distance: 836.1829637071355
Gap: 0.11024768060785468%


#### Duration
Average serial duration: 48.086801528930664

### OR-Tools

In [18]:
tour_str = """
0 68 74 61 28 62 33 51 6 0
0 73 1 43 23 56 41 42 64 22 0
0 38 65 10 31 55 25 50 3 0
0 11 66 59 14 35 0
0 53 19 54 13 57 15 37 5 29 0
0 26 12 58 72 39 9 40 0
0 17 44 32 18 24 49 63 16 0
0 7 8 52 27 45 30 2 0
0 21 69 71 60 70 20 36 47 48 0
0 75 4 34 46 67 0"""

sub_tours = [[int(n) for n in r.split()[1:-1]] for r in tour_str.split("\n") if r]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 10
Total clients visited: 75
Total distance: 890.7836838490479
Gap: 6.647204129365212%


#### Duration
Average serial duration: 60.0

### RL current solution

In [19]:
tour = [ 0, 16, 49, 24, 18, 50, 44,  3, 51,  0, 63, 23, 56, 41, 42, 64, 43,  1,
        73,  0, 33, 62, 22, 61, 28,  2, 68,  0, 17, 40, 32,  9, 12,  0,  6, 74,
        21, 69, 36, 47, 48, 30,  0, 39, 25, 55, 31, 10, 72, 58, 26,  0,  5, 37,
        71, 60, 70, 20, 15, 57, 29, 45,  0, 38, 65, 66, 11, 53, 35,  0, 27, 13,
        54, 19, 59, 14,  7,  0,  4, 52,  8, 46, 34, 75,  0, 67,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 11
Total clients visited: 75
Total distance: 867.158691902471
Gap: 3.818751628086402%


#### Duration
Average serial duration: 1.7034242153167725

In [20]:
tour = [ 0, 16, 49, 24, 18, 50, 44,  3, 51,  0, 63, 23, 56, 41, 42, 64, 43,  1,
        73,  0, 33, 62, 22, 61, 28,  2, 68,  0, 17, 40, 32,  9, 12,  0,  6, 74,
        21, 69, 47, 36, 48, 30,  0, 39, 55, 25, 31, 10, 72, 58, 26,  0,  5, 37,
        71, 60, 70, 20, 15, 57, 29, 45,  0, 38, 65, 66, 11, 53, 35,  0, 13, 27,
        54, 19, 59, 14,  7,  0,  8, 52,  4, 34, 46, 75,  0, 67,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 11
Total clients visited: 75
Total distance: 907.4528226333771
Gap: 8.64288173193604%


#### Duration
Average serial duration: 11.8

## CMT3

In [21]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT3.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [22]:
solution = """92 37 98 100 91 16 86 38 44 14 42 43 15 57 2 58
6 96 99 59 93 85 61 17 45 84 5 60 89
27 69 1 70 30 20 66 32 90 63 10 62 88 31
21 72 75 56 39 67 23 41 22 74 73 40
18 83 8 46 47 36 49 64 11 19 48 82 7 52
94 95 97 87 13
28 12 80 68 29 24 54 55 25 4 26 53
76 77 3 79 78 34 35 65 71 9 51 81 33 50"""
# Cost 826.137

In [23]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [24]:
len(sub_tours)

8

In [25]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

826.136972955021

### LKH3 Solution

In [26]:
tour = [0, 12, 80, 68, 24, 29, 34, 78, 79, 3, 77, 76, 28, 0, 
        94, 95, 59, 98, 85, 93, 99, 96, 6, 0, 92, 37, 100, 
        91, 44, 14, 38, 86, 16, 61, 5, 60, 89, 0, 69, 1, 
        70, 30, 20, 66, 65, 71, 35, 9, 51, 81, 33, 50, 0, 
        58, 2, 75, 56, 23, 67, 39, 25, 55, 4, 54, 26, 0, 
        53, 40, 21, 73, 72, 74, 22, 41, 57, 15, 43, 42, 87, 
        97, 13, 0, 88, 62, 11, 19, 47, 48, 82, 7, 52, 0, 27, 
        31, 10, 32, 90, 63, 64, 49, 36, 46, 8, 45, 17, 84, 83, 18, 0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 100
Total distance: 829.4473653779123
Gap: 0.40070745303292493%


#### Duration
Average serial duration: 14.121454954147339

### OR-Tools

In [27]:
tour_str = """
0 69 70 30 10 62 63 90 32 20 66 65 71 51 50 0
0 53 40 21 73 72 74 75 22 41 15 43 14 100 37 92 95 0
0 27 31 88 19 11 64 49 36 47 46 45 17 84 60 18 0
0 6 98 85 61 16 86 38 44 91 42 57 2 58 0
0 28 76 29 24 54 55 25 39 67 23 56 4 26 0
0 52 7 48 82 8 83 5 93 99 59 96 89 0
0 94 97 87 13 0
0 1 33 81 9 35 34 78 79 3 77 68 80 12 0"""

sub_tours = [[int(n) for n in r.split()[1:-1]] for r in tour_str.split("\n") if r]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 100
Total distance: 875.090483205157
Gap: 5.925592468647615%


#### Duration
Average serial duration: 60.0

### RL current solution

In [28]:
tour = [  0,  58,   2,  57,  15,  43,  42,  14,  38,  44,  86,  16, 100,  37,
         97,   6,   0,  40,  21,  73,  74,  41,  22,  75,  56,  23,  67,  39,
         72,   0,  53,  26,   4,  25,  55,  54,  24,  29,  80,  68,  12,  28,
          0,  13,  87,  92,  98,  91,  85,  93,  59,  95,  94,   0,  77,   3,
         79,  78,  34,  35,  65,  71,  66,   9,  81,  33,   1,   0,  96,  99,
         61,  17,  84,   5,  60,  83,  45,  46,   8,  18,  89,   0,  76,  50,
         51,  20,  30,  70,  10,  32,  90,  63,  64,  49,  36,  47,   7,   0,
         27,  69,  31,  88,  62,  11,  19,  48,  82,  52,   0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 100
Total distance: 882.5217380732901
Gap: 6.825110963934426%


#### Duration
Average serial duration: 1.9495947360992432

In [29]:
tour = [  0,  58,   2,  57,  15,  43,  42,  14,  38,  44,  86,  16, 100,  37,
         97,   6,   0,  40,  21,  73,  74,  41,  22,  75,  56,  23,  67,  39,
         72,   0,  26,  53,   4,  25,  55,  54,  24,  29,  68,  80,  12,  28,
          0,  13,  87,  92,  98,  91,  85,  93,  59,  95,  94,   0,  77,   3,
         79,  34,  78,  35,  65,  71,  66,   9,  81,  33,   1,   0,  96,  99,
         61,  17,  84,   5,  60,  45,  83,  46,   8,  18,  89,   0,  76,  50,
         51,  20,  30,  70,  10,  32,  90,  63,  64,  49,  36,  47,   7,   0,
         27,  69,  31,  88,  62,  11,  19,  48,  82,  52,   0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 100
Total distance: 915.3050540539109
Gap: 10.793377371786592%


#### Duration
Average serial duration: 17.5

## CMT11

In [30]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT11.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [31]:
solution = """119 81 112 84 117 113 83 108 118 18 114 90 91 89 85 86 111 82
88 2 1 3 4 5 6 7 9 10 11 15 14 13 12 8
17 16 19 25 22 24 27 33 30 31 34 36 29 35 32 28 26 23 20 21 109
40 43 45 48 51 50 49 46 47 44 41 42 39 38 37 95
52 54 57 59 65 61 62 64 66 63 60 56 58 55 53 100
106 73 76 68 77 79 80 78 75 72 74 71 70 69 67 107
120 105 102 101 99 104 103 116 98 110 115 97 94 96 93 92 87"""
# Cost 1042.12

In [32]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [33]:
len(sub_tours)

7

In [34]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

1042.1150274399192

### LKH3 Solution

In [35]:
tour = [0, 82, 111, 86, 85, 89, 91, 90, 114, 18, 118, 108, 83, 
        113, 117, 84, 112, 81, 119, 0, 120, 105, 102, 101, 
        99, 104, 103, 116, 98, 110, 115, 97, 94, 96, 93, 92, 
        87, 0, 109, 21, 20, 23, 26, 28, 32, 35, 29, 36, 34, 
        31, 30, 33, 27, 24, 22, 25, 19, 16, 17, 0, 100, 52, 
        54, 57, 59, 65, 61, 62, 64, 66, 63, 60, 56, 58, 55, 
        53, 0, 107, 67, 69, 70, 71, 74, 75, 72, 78, 80, 79, 
        77, 68, 76, 73, 106, 0, 40, 43, 45, 48, 51, 50, 49, 
        46, 47, 44, 41, 42, 39, 38, 37, 95, 0, 8, 12, 13, 14, 
        15, 11, 10, 9, 7, 6, 5, 4, 3, 1, 2, 88, 0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 7
Total clients visited: 120
Total distance: 1042.1151057480174
Gap: 7.5143430542645274e-06%


#### Duration
Average serial duration: 38.99067497253418

### OR-Tools

In [36]:
tour_str = """
0 50 59 65 61 62 64 66 63 60 58 56 55 53 54 57 52 79 0
0 46 49 29 32 35 36 34 30 33 27 31 28 25 24 22 19 16 17 20 23 26 21 0
0 39 42 38 37 41 44 47 51 48 45 43 40 80 78 74 0
0 109 115 110 98 68 73 76 77 72 75 71 67 69 70 0
0 118 8 12 13 14 15 11 10 9 7 6 5 4 3 1 2 0
0 85 112 84 117 113 83 108 18 114 90 91 94 97 116 100 103 107 104 99 101 0
0 120 119 81 82 88 111 86 87 89 92 93 96 95 102 105 106 0"""

sub_tours = [[int(n) for n in r.split()[1:-1]] for r in tour_str.split("\n") if r]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 7
Total clients visited: 120
Total distance: 1177.903795268736
Gap: 13.030113207598424%


#### Duration
Average serial duration: 60.0

### RL current solution

In [37]:
tour = [  0,  54,  61,  57,  59,  65,  62,  64,  66,  63,  60,  58,  53,  55,
         56,  52,  79,  76,   0,  68,  73,  77,  78,  80,  72,  74,  71,  75,
         70,  69,  67, 103,   0, 120, 119,  81,  82,  88,  86,  87,  90,  89,
         92,  95,   0, 117,  83, 113, 112,  84, 111,  85,  18, 114, 108,  97,
          0,  10,   2,   1,   3,   5,   4,  11,  15,   9,   6,   7,  14,  13,
         12,   8, 118,   0,  17,  16,  19,  25,  22,  24,  27,  20,  23,  21,
         26,  28,  30,  33,  31,  32,  35,  34,  36,  29, 109,   0,  91,  39,
         37,  41,  38,  44,  46,  49,  47,  42,  48,  50,  51,  45,  43,  40,
        107,   0,  93,  96,  94, 115, 110, 100,  99,  98, 116, 104, 101, 102,
        106, 105,   0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 120
Total distance: 1207.7441755500604
Gap: 15.893557212875919%


#### Duration
Average serial duration: 2.9753198623657227

In [38]:
tour = [  0,  54,  61,  57,  59,  65,  62,  64,  66,  63,  60,  58,  53,  55,
         56,  52,  79,  76,   0,  68,  73,  77,  78,  80,  72,  74,  71,  75,
         70,  69,  67, 103,   0, 120, 119,  81,  82,  88,  86,  87,  90,  89,
         92,  95,   0, 117,  83, 113, 112,  84, 111,  85,  18, 114, 108,  97,
          0,  10,   2,   1,   3,   5,   4,  11,   9,  15,   6,   7,  14,  13,
         12,   8, 118,   0,  17,  16,  19,  24,  25,  22,  27,  20,  23,  21,
         26,  28,  30,  33,  31,  32,  35,  34,  36,  29, 109,   0,  91,  39,
         37,  41,  38,  44,  46,  49,  47,  42,  48,  50,  51,  45,  43,  40,
        107,   0,  93,  94,  96, 110, 115,  99, 100,  98, 116, 102, 101, 104,
        106, 105,   0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 8
Total clients visited: 120
Total distance: 1221.8212180049359
Gap: 17.244371862335246%


#### Duration
Average serial duration: 21.8

---

## CMT1 Overfit

In [39]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT1.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [40]:
solution = """46 5 49 10 39 33 45 15 44 37 12
11 2 29 21 16 50 34 30 9 38
8 26 31 28 3 36 35 20 22 1 32
27 48 23 7 43 24 25 14 6
18 13 41 40 19 42 17 4 47"""
# Cost 524.611

In [41]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [42]:
len(sub_tours)

5

In [43]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

524.6111466425073

### RL current solution

In [44]:
tour = [ 0, 17, 42, 19, 40, 41, 13, 18,  4, 47,  0,  6, 14, 25, 24, 43,  7, 23,
        48, 27,  0, 12, 37, 44, 15, 45, 33, 39, 10, 49,  5, 46,  0,  1,  8, 26,
        31, 28, 22,  3, 36, 35, 20, 29, 21,  0, 32,  2, 16, 50, 34, 30,  9, 38,
        11,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 538.4425856141712
Gap: 2.636512597985129%


#### Duration
Average serial duration: 1.6

## CMT1 Reuse

In [45]:
data = parse_file(path("cvrplib/Christofides, Mingozzi and Toth (1979)/CMT1.vrp"))
coords = {k: np.array(v) for k, v in data["node_coord_section"].items()}
demand = data["demand_section"]

#### Solution File

In [46]:
solution = """46 5 49 10 39 33 45 15 44 37 12
11 2 29 21 16 50 34 30 9 38
8 26 31 28 3 36 35 20 22 1 32
27 48 23 7 43 24 25 14 6
18 13 41 40 19 42 17 4 47"""
# Cost 524.611

In [47]:
sub_tours = [[int(v) for v in st.split()]for st in solution.split("\n")]

In [48]:
len(sub_tours)

5

In [49]:
for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")

baseline = np.sum([compute_cost(r, coords) for r in sub_tours])
baseline

524.6111466425073

### RL current solution

In [50]:
tour = [ 0,  6, 14, 25, 24, 43,  7, 23, 48, 27,  0, 18, 13, 41, 40, 19, 42,  4,
        47,  0, 12, 17, 37, 44, 15, 45, 33, 39, 10, 49,  5,  0, 38,  9, 30, 34,
        50, 21, 29, 16,  2, 11,  0, 32, 20, 35, 36,  3, 28, 31, 26,  8, 22,  1,
        46,  0]

tour = str(tour)
sub_tours = [[int(n.strip()) for n in st.replace("[0, ", "").replace(", 0]", "").split(",") if n] for st in tour.split(" 0,") if st]

for r in sub_tours:
    d = compute_demand(r, demand)
    if d > data["capacity"]:
        print(f"!!!! Demand ({d}) exceeds capacity ({data['capacity']}) for subtour:\n\t{r}")
        
print(f"Number of sub-tours: {len(sub_tours)}")
print(f"Total clients visited: {np.sum([len(r) for r in sub_tours])}")
total_distance = np.sum([compute_cost(r, coords) for r in sub_tours])
print(f"Total distance: {total_distance}")
print(f"Gap: {(total_distance - baseline) / baseline * 100}%")

Number of sub-tours: 5
Total clients visited: 50
Total distance: 531.8638624750281
Gap: 1.3824936581957814%


#### Duration
Average serial duration: 1.4

---